In [1]:
#First Notebook

In [2]:
!pip install boto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.2 MB/s eta 0:00:0000:010:01


In [3]:
!pip install opensearch-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.0 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.


In [4]:
!pip install requests-aws4auth

In [17]:
!pip install awswrangler

In [18]:
!pip install jsonpath-ng

In [5]:
#Imports
import json
import pandas as pd
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

In [10]:
def os_connection():
    host = 'search-dbms-twitter-domain-xpmt2uaqemhxgnw3ucwekwhvhq.us-east-2.es.amazonaws.com'
    region = 'us-east-2'

    service = 'es'
    credentials = boto3.Session().get_credentials()
    awsauth = AWS4Auth(region = region, service = service, session_token = credentials.token, refreshable_credentials = credentials)

    os = OpenSearch(
        hosts = [{'host': host, 'port': 443}],
        http_auth = awsauth,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection)
    
    return os

In [11]:
os = os_connection()

In [9]:
#Match Query: This type of query matches documents based on a specific field and value.
query = {
    "query": {
        "match": {
            "field_name": "search_value"
        }
    }
}

#Term Query: This query matches documents where a specified field contains a particular term.
query = {
    "query": {
        "term": {
            "field_name": "search_term"
        }
    }
}

#Range Query: Use this query to match documents within a specified range.
query = {
    "query": {
        "range": {
            "date_field": {
                "gte": "2024-01-01",
                "lte": "2024-03-31"
            }
        }
    }
}

#Bool Query: Combine multiple queries using boolean logic (must, should, must_not).
query = {
    "query": {
        "bool": {
            "must": [
                {"match": {"field1": "value1"}},
                {"term": {"field2": "value2"}}
            ],
            "should": [
                {"match": {"field3": "value3"}}
            ],
            "must_not": [
                {"term": {"field4": "value4"}}
            ]
        }
    }
}

#Aggregation Query: Perform aggregations to summarize data.
query = {
    "aggs": {
        "agg_name": {
            "aggregation_type": {
                "field": "field_name"
            }
        }
    }
}

## Adding a record

In [15]:
os.index(index = "twitter-data-user",body = {"id_str": "Kalal1"})

{'_index': 'twitter-data-user',
 '_id': 'jjTXB48B7iUTFS3gcQb7',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 32676,
 '_primary_term': 1}

## Deleting a record

In [14]:
query = {
    "query": {
        "match": {
            "id_str": "Kalal1"
        }
    }
}
os.delete_by_query(index="twitter-data-user", body=query)

{'took': 35,
 'timed_out': False,
 'total': 1,
 'deleted': 1,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

## Updating a record by query

In [16]:
# Define the index name
index_name = "twitter-data-user"

# Define the update query
update_query = {
    "script": {
        "source": "ctx._source.id_str = params.new_value",
        "params": {
            "new_value": "Katik"
        }
    },
    "query": {
        "match": {
            "id_str": "Kalal1"
        }
    }
}

# Update documents in the index using the update by query API
response = os.update_by_query(index=index_name, body=update_query)

# Print the response
print(response)

{'took': 5933, 'timed_out': False, 'total': 1, 'updated': 1, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}


## Searching a record using AWS Wrangler

In [22]:
import awswrangler as wr
import pandas as pd

In [23]:
wrangler = wr.opensearch.connect(host = 'https://search-dbms-twitter-domain-xpmt2uaqemhxgnw3ucwekwhvhq.us-east-2.es.amazonaws.com', port = 443, username = 'admin-shreyash', password = 'SskRutgers@123')

In [24]:
query ='''
{
  "query": {
    "match": {
      "id_str": "Katik"
    }
  }
}
'''

In [26]:
df = wr.opensearch.search(client = wrangler, index = 'twitter-data-user', 
                          search_body = query)
df

,_id,id_str
0,jjTXB48B7iUTFS3gcQb7,Katik
